In [1]:
import pandas as pd 
import numpy as np
import sklearn as sk 
import matplotlib.pyplot as plt
import json
import Module.Conn as conn 
import Module.GetDataSet as getdataset 
from Module.DotClass import *
import random
random.seed(0)

In [2]:
#read data
standard_df=getdataset.get_standard_dataset()
# work_df=getdataset.get_work_dataset()

NameError: name 'getdataset' is not defined

In [4]:
# 五折训练集和测试集数据
from sklearn.model_selection import train_test_split
idlist=list(set(standard_df.id))
idlist_train,idlist_test = train_test_split(idlist,train_size=0.8,random_state =0)
train_df=standard_df[standard_df.id.isin(idlist_train)]
test_df=standard_df[standard_df.id.isin(idlist_test)]

In [5]:
# 先看传统机器学习算法
#构造特征表
total_stroke=[]
total_simple_stroke=[]
width_list=[]
length_list=[]
ratio_list=[]
label_list=[]
dot_count_list=[]
dotdist_list=[]
angle_list=[]
data=standard_df
for k in range(len(data)):
    t=Stroke(data['trajectory'].iloc[k],data['label'].iloc[k])
    total_stroke.append(t)
    total_simple_stroke.append(t.simple_dot)
    dot_count_list.append(len(t.simple_dot))
    dotdist_list.append(t.dotdist_list)
#     if t.angle_list
    angle_list.append(t.angle_list)
    k=t.get_width_length_ratio()
    width_list.append(k[0])
    length_list.append(k[1])
    ratio_list.append(k[2])
    label_list.append(t.get_label())
dotdist_list_count=[len(i) for i in dotdist_list]
dotdist_list_max=[max(i) for i in dotdist_list]
dotdist_list_min=[min(i) for i in dotdist_list]
dotdist_list_median=[np.median(i) for i in dotdist_list] 
angle_list_max=[max(i) for i in angle_list]
angle_list_min=[min(i) for i in angle_list]
angle_list_median=[np.median(i) for i in angle_list] 
angle_list_90=[close90(i) for i in angle_list] 


E:\科研\规范汉字评价模型\code\Module\DotClass.py:143: RuntimeWarning: invalid value encountered in arccos
  arccos = np.arccos(vector_dot_product / (np.linalg.norm(v1) * np.linalg.norm(v2)))


In [6]:
df=pd.DataFrame([label_list,data.id,width_list,length_list,ratio_list,dotdist_list_count,dotdist_list_max,dotdist_list_min,dotdist_list_median,angle_list_max,angle_list_min,angle_list_median,angle_list_90]).T
df.columns=["label","id","width","height","ratio_w_h","dotdist_count","dotdist_max","dotdist_min","dotdist_median","angle_max","angle_min","angle_median","angle_90"]
df=df.fillna(0)
train_ml=df[df.id.isin(idlist_train)]
test_ml=df[df.id.isin(idlist_test)]

In [10]:
# 多个分类器的比较
# https://blog.csdn.net/TiffanyRabbit/article/details/76574009
from sklearn.ensemble import RandomForestClassifier
import sklearn.tree as sk_tree
import sklearn.linear_model as sk_linear
import sklearn.naive_bayes as sk_bayes
import sklearn.neighbors as sk_neighbors
import sklearn.svm as sk_svm
import sklearn.neural_network as sk_nn
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut


def get_acc_list(colist,name):
    X=mytotal[colist]
    Y=mytotal[name]
    method_list=["随机森林模型"]
    acc_auc_list=[]
    def train_and_score(colist,name,model,modelname):
        X=mytotal[colist]
        Y=mytotal[name]
        print("Y",len(Y),name)
        acc=cross_val_score(model,X,Y,cv=5)
        print(acc) 
        acc=np.mean(acc)
        print('{}评价:'.format(modelname),acc)
        return acc 


    model = RandomForestClassifier() 
#     gb_param_grid = {'criterion' : ["gini", "entropy"],
#                   'n_estimators': [50,100,200],
#                   }
#     grid_search = GridSearchCV(model,param_grid = gb_param_grid, cv=5,scoring="accuracy", n_jobs= -1, verbose = 1)
#     grid_result=grid_search.fit(X,Y)
#     params=grid_search.best_params_ 
    model = RandomForestClassifier(criterion="entropy", n_estimators=50,random_state=0)
    acc_auc_list.append(train_and_score(colist,name,model,method_list[0]))
#     print(params)


    return acc_auc_list

In [11]:
mytotal=df
drop_list=["label","id"]
colist=[]
for i in mytotal.columns:
    if  i not in drop_list: 
        colist.append(i)

total=[]
total.append(get_acc_list(colist,name='label'))
method_list=["随机森林模型"]
t=["label"]
d=pd.DataFrame(total,index=t,columns=method_list)


Y 19657 label


d:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


[0.86164802 0.87131231 0.87331468 0.86695497 0.85627067]
随机森林模型评价: 0.8659001292126414


In [12]:
from sklearn.metrics import accuracy_score,f1_score

model = RandomForestClassifier(criterion="entropy", n_estimators=50,random_state=0)
# 相同的训练集和测试集上模型准确率
#关于macro-f1 https://blog.csdn.net/qq_43190189/article/details/105778058
x_train_ml=train_ml[colist]
y_train_ml=train_ml['label']
x_test_ml=test_ml[colist]
y_test_ml=test_ml['label']
model.fit(x_train_ml[colist],y_train_ml)
y_train_ml_res=model.predict(train_ml[colist])
y_test_ml_res=model.predict(test_ml[colist])
train_ml_acc=accuracy_score(y_train_ml_res,y_train_ml)
test_ml_acc=accuracy_score(y_test_ml_res,y_test_ml)
train_ml_f1=f1_score(y_train_ml_res,y_train_ml,average='macro')
test_ml_f1=f1_score(y_test_ml_res,y_test_ml,average='macro')
print('train acc:',train_ml_acc,'test acc:',test_ml_acc)
print('train macro f1:',train_ml_f1,'test macro f1:',test_ml_f1)

train acc: 0.9998730642294998 test acc: 0.8687516021532941
train macro f1: 0.9999331771899063 test macro f1: 0.6137876170933183


In [14]:
import pickle
#save model
f = open('model/rfc.pickle','wb')
pickle.dump(model,f)
f.close()
#load model
f = open('model/rfc.pickle','rb')
rfc1 = pickle.load(f)
f.close()
y_test_ml_res2=rfc1.predict(x_test_ml)
test_ml_acc=accuracy_score(y_test_ml_res2,y_test_ml)
print('test acc:',test_ml_acc)

test acc: 0.8687516021532941


In [70]:
# 跑CNN模型，得到f1 结果
import json 
with open('data/predict_topk_standard.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    print('这是文件中的json数据：',json_data)
    print('这是读取到文件数据的数据类型：', type(json_data))
# json.load()

这是文件中的json数据： 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [72]:
# 错误类型找到字 
model_res=pd.DataFrame(json_data)
# model_res.join(standard_df,on=['id','seq'],how='right')
# model_res.predict
standard_df1=standard_df.copy()
standard_df1['key']=1 
standard_df1['num']=standard_df1.apply(lambda x:standard_df1[standard_df1.id==x.id].id.count(),axis=1)
standard_df1['predict']=model_res.predict
standard_df1['seq_real']=standard_df1.seq.copy()
standard_df1

,trajectory,label,id,seq,ch,key,num,predict,seq_real
0,"[{""ab_x"": 17.202380620361396, ""ab_y"": 52.13335...",横折折折钩/横撇弯钩,61c2b7e0c0766b5221b5394c,1,阿,1,7,"[{'label': '横折折折钩/横撇弯钩', 'logit': 0.9969156980...",1
1,"[{""ab_x"": 23.352776399687997, ""ab_y"": 50.14687...",竖,61c2b7e0c0766b5221b5394c,2,阿,1,7,"[{'label': '竖', 'logit': 0.8693052530288696}, ...",2
2,"[{""ab_x"": 33.95378601704408, ""ab_y"": 54.222562...",横,61c2b7e0c0766b5221b5394c,3,阿,1,7,"[{'label': '横', 'logit': 0.9999420642852783}, ...",3
3,"[{""ab_x"": 38.70569152568106, ""ab_y"": 60.340967...",竖,61c2b7e0c0766b5221b5394c,4,阿,1,7,"[{'label': '竖', 'logit': 0.9654157161712646}, ...",4
4,"[{""ab_x"": 39.31614391218443, ""ab_y"": 61.314162...",横折,61c2b7e0c0766b5221b5394c,5,阿,1,7,"[{'label': '横折', 'logit': 0.9899035692214966},...",5
...,...,...,...,...,...,...,...,...,...
19652,"[{""ab_x"": 35.76188390120487, ""ab_y"": 78.036063...",横,6107cceea11a58c6dbb96b42,7,做,1,11,"[{'label': '横', 'logit': 0.9996292591094971}, ...",7
19653,"[{""ab_x"": 54.50604757126203, ""ab_y"": 41.264314...",撇,6107cceea11a58c6dbb96b42,8,做,1,11,"[{'label': '撇', 'logit': 0.9935024380683899}, ...",8
19654,"[{""ab_x"": 50.21932889659931, ""ab_y"": 62.015418...",横,6107cceea11a58c6dbb96b42,9,做,1,11,"[{'label': '横', 'logit': 0.9998260140419006}, ...",9
19655,"[{""ab_x"": 56.88941086381243, ""ab_y"": 59.053125...",撇,6107cceea11a58c6dbb96b42,10,做,1,11,"[{'label': '撇', 'logit': 0.999316930770874}, {...",10


In [74]:
pred=[]
for i in range(len(standard_df1)):
    p=standard_df1.iloc[i]['predict'][0]['label']
    pred.append(p)
   
    
standard_df1['pred']=pred

In [77]:
t=standard_df1[standard_df1.id.isin(idlist_test)]
y=t.label
yhat=t.pred
acc=accuracy_score(yhat,y)
f1=f1_score(yhat,y,average='macro')
print('train acc:',acc,'test acc:',f1)
# print('train macro f1:',train_ml_f1,'test macro f1:',test_ml_f1)

train acc: 0.9427122940430925 test acc: 0.917856352232208


In [ ]:
#封装几种算法，数据预先处理
model_res

In [ ]:
# 错误的数据集合上找到的错误。 
df

In [ ]:
# 找到错误的比例。
# 随机找到一个笔画

In [ ]:
# 笔画分类验证，精确到类别


In [ ]:
# 构造错误分类数据集


In [ ]:
# 构造对应关系的模型
